## Notebook for making predictions with deleted constant y samples


In [11]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [12]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [13]:
X_A_train = dm.train_a.iloc[:, 2:-1]
Y_A_train = dm.train_a.iloc[:, 0]

X_B_train = dm.train_b.iloc[:, 2:-1]
Y_B_train = dm.train_b.iloc[:, 0]

X_C_train = dm.train_c.iloc[:, 2:-1]
Y_C_train = dm.train_c.iloc[:, 0]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

In [14]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

In [20]:
X_B_train.shape

(104574, 25)

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [15]:
catModel_a = CatBoostRegressor(objective="MAE", learning_rate=0.10931167365445349, colsample_bylevel=0.09963123954233088, max_depth=15, boosting_type="Plain", bootstrap_type="MVS")
catModel_b = CatBoostRegressor(objective="MAE", learning_rate=0.10931167365445349, colsample_bylevel=0.09963123954233088, max_depth=15, boosting_type="Plain", bootstrap_type="MVS")
catModel_c = CatBoostRegressor(objective="MAE", learning_rate=0.10931167365445349, colsample_bylevel=0.09963123954233088, max_depth=15, boosting_type="Plain", bootstrap_type="MVS")

In [16]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

0:	learn: 602.3505804	total: 211ms	remaining: 3m 31s
1:	learn: 558.1910742	total: 228ms	remaining: 1m 53s
2:	learn: 531.0360512	total: 236ms	remaining: 1m 18s
3:	learn: 488.0543249	total: 404ms	remaining: 1m 40s
4:	learn: 484.0774485	total: 414ms	remaining: 1m 22s
5:	learn: 483.8803163	total: 421ms	remaining: 1m 9s
6:	learn: 483.7085573	total: 426ms	remaining: 1m
7:	learn: 483.5579622	total: 432ms	remaining: 53.6s
8:	learn: 446.0512664	total: 687ms	remaining: 1m 15s
9:	learn: 418.8278368	total: 757ms	remaining: 1m 14s
10:	learn: 400.7744767	total: 765ms	remaining: 1m 8s
11:	learn: 388.6313416	total: 773ms	remaining: 1m 3s
12:	learn: 388.5284179	total: 779ms	remaining: 59.1s
13:	learn: 383.1979321	total: 786ms	remaining: 55.4s
14:	learn: 362.8564064	total: 817ms	remaining: 53.7s
15:	learn: 348.4835576	total: 824ms	remaining: 50.7s
16:	learn: 348.4046038	total: 830ms	remaining: 48s
17:	learn: 348.2370268	total: 836ms	remaining: 45.6s
18:	learn: 348.1854613	total: 841ms	remaining: 43.4s
1

## Preds

In [8]:
pd.set_option('display.max_columns', None)
X_B_test.head(3)

,absolute_humidity_2m:gm3,ceiling_height_agl:m,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,fresh_snow_1h:cm,fresh_snow_24h:cm,is_in_shadow:idx,msl_pressure:hPa,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_w_1000hPa:ms,date_forecast,month,season,hours,sum_rad:W,rad_cloud_ratio
0,4.30,912.299988,1059.750000,0.0,271.650024,0.0,0.0,1.0,1013.775024,0.0,1006.674988,0.0,0.0,80.250000,1012.974976,0.0,0.0,-10.54025,75.000000,31107.599609,3.950,0.0,2023-05-01 00:00:00,0.522008,0.809017,0,0.0,0.0
1,4.25,1875.195190,1073.699951,0.0,271.450012,0.0,0.0,1.0,1013.250000,0.0,1006.174988,0.0,0.0,79.800003,1012.474976,0.0,0.0,-7.89400,77.500000,30409.699219,3.825,0.0,2023-05-01 01:00:00,0.522008,0.809017,1,0.0,0.0
2,4.15,1765.900024,1200.099976,0.0,271.049988,0.0,0.0,1.0,1012.700012,0.0,1005.650024,0.0,0.0,78.000000,1011.950012,0.0,0.0,-3.81500,88.099998,31342.650391,3.650,0.0,2023-05-01 02:00:00,0.522008,0.809017,2,0.0,0.0


In [17]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

In [18]:
df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("sub43.csv")

df.shape

(2160, 1)